# Le problème SAT 

## Objet du problème

$X=\{p_1,\ldots,p_{n}\}$ est un ensemble de variables propositionnelles. L'expression $p_i$ (resp. $\lnot p_i$) est appelée un *littéral positif* (resp. un *littéral négatif*). Une *clause* est une disjonction de littéraux. Une conjonction de clauses ($\varphi=c_1 \land \ldots \land c_k$ où $c_i=\ell_{i,1}\lor\ldots\lor\ell_{i,h_i}$ avec $\ell_{i,j}=p_{i,j}$ ou $\lnot p_{i,j}$, $p_{i,j}\in X$) est appelée une FNC ou *forme normale conjonctive*. 

On désire déterminer la *satisfiabilité*  d'une FNC $\varphi$ ; c.-à-d. décider s'il existe une affectation des $p_i$ rendant $\varphi$ vraie. En cas de succés, il n'est pas plus difficile de calculer une telle affectation. Ce problème est NP-complet ; il faut donc s'attendre à une complexité importante.

## Algorithme de Davis, Putnam, Logemann et Loveland (DPLL)

DPLL($\varphi$)    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**tant que** $\varphi$ contient une clause unitaire $\{\ell\}$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;affecter *Vrai* à $\ell$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;supprimer de $\varphi$ les clauses contenant $\ell$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;supprimer $\lnot\ell$ des clauses de $\varphi$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**si** $\emptyset\in\varphi$ **alors** **retourner** *Faux*  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**tant que** $\varphi$ contient un littéral pur $\ell$ ($\lnot\ell$ n'est pas dans $\varphi$)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;affecter *Vrai* à $\ell$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;supprimer de $\varphi$ les clauses contenant $\ell$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**si** $\varphi=\emptyset$ **alors** **retourner** *Vrai*  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;choisir un littéral $\ell$ apparaissant dans $\varphi$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**retourner** DPLL($\varphi\cup\{\{\ell\}\}$) **ou** DPLL($\varphi\cup\{\{\lnot\ell\}\}$)

### Implémentation

On représente le littéral $p_i$ par l'entier $i$ et $\lnot p_i$ par $-i$. Une clause est vue comme un ensemble (type `set`) de littéraux et une formule comme une liste (type `list`) de clauses.

In [1]:
from copy import deepcopy

# choisir un élément d'un ensemble
def choisir(s):
    e = s.pop()
    s.add(e)
    return e
    
def dpll(f,n):

    # solution
    d = {i : None for i in range(1, n + 1)}
    
    # littéraux présents dans f
    lit = set.union(*f)

    # affecter Vrai au littéral l
    def affecter(l):
        if l > 0: 
            d[l] = True
        else:
            d[-l] = False
          
    def dpllrec():

        nonlocal f, lit
        
        cont = True
        # tant que f a une clause contenant un unique littéral
        while cont:
            try:
                # rechercher une clause contenant un unique littéral
                l = choisir(next(filter(lambda c: len(c) == 1, f)))
                # affecter la variable correspondante
                affecter(l)
                # supprimer de f les clauses contenant l
                f = [c for c in f if l not in c]
                # supprimer non(l) de toutes les clauses
                for c in f:
                    if -l in c:
                        c.remove(-l)
                # mettre à jour lit
                lit -= {l,-l}
            except StopIteration:
                cont = False
        # si f contient un clause vide, f n'est pas satisfiable
        if set() in f:
            return False
        cont = True
        # tant que f a un littéral pur
        while cont:
            try:
                # rechercher un littéral pur
                l = next(filter(lambda l: l in lit and -l not in lit, lit))
                # affecter la variable correspondante
                affecter(l)
                # supprimer de f les clauses contenant l
                f = [c for c in f if l not in c]
                # mettre à jour lit
                lit.remove(l)
            except StopIteration:
                cont = False
        if f == []:
            return True
        if lit == set():
            return False
        l = choisir(lit)
        f_, lit_ = deepcopy((f, lit))
        f.append({l})
        if dpllrec():
            return True
        else:
            f , lit = f_, lit_
            f.append({-l})
            lit |= {-l}
            return dpllrec()
     
    # rq: si f n'est pas satisfiable retourne None 
    if dpllrec():
        return d

### Test

In [2]:
dpll([{5,-2,-3},{3,-2,-4},{2,-3},{-3,-4},{4},{-5,-2,-3,1}],5)

{1: True, 2: False, 3: False, 4: True, 5: None}

### Fonctions utiles

Pour une liste ou un ensemble de litteraux `l`, chacune des 3 fonctions suivantes ajoute une ou des clause(s) à une formule `f`.

In [3]:
# - Au moins un littéral de l est Vrai -
def auMoinsUn(f,l):
    f.append(set(l))
    
# - Au plus un littéral de l est Vrai -
def auPlusUn(f,l):
    for p in l:
        for q in l:
            if p < q:
                f.append({- p, - q})

# - Exactement un littéral de l est Vrai -
def exactementUn(f,l):
    auMoinsUn(f,l)
    auPlusUn(f,l)

In [4]:
def vrai(f,p):
    f.append({p})
    
def faux(f,p):
    f.append({- p})

# p => q où p est un littéral et q est soit un littéral, soit une clause 
def implique(f,p,q):
    if isinstance(q,int):
        f.append({- p, q})
    else:
        assert isinstance(q,set)
        f.append({- p} | q)

## 1. Le [problème du zèbre](https://fr.wikipedia.org/wiki/Int%C3%A9gramme)

Ce problème (parfois attribué à Einstein ou à Lewis Carroll) s'énonce ainsi :

Cinq hommes de nationalités et de professions différentes habitent avec leur animal de compagnie cinq maisons de couleurs différentes où ils boivent leur boisson préférée. On dispose de 14 indices :

1. L'anglais habite la maison rouge ;
2. l'espagnol a un chien ;
3. l'islandais est ingénieur ;
4. la maison verte sent bon le café ;
5. la maison verte est située immédiatement à gauche de la blanche ;
6. le sculpteur possède un âne ;
7. le diplomate habite la maison jaune ;
8. le norvégien habite la première maison à gauche ;
9. le médecin est voisin du propriétaire du renard ;
10. la maison du diplomate est voisine de celle où il y a un cheval ;
11. on boit du lait dans la maison du milieu ;
12. le slovène boit du thé ;
13. le violoniste boit du jus d'orange ;
14. le norvégien demeure à côté de la maison bleue.

Question : qui possède un zèbre ?

### Solution

In [5]:
# les différents attributs
nationalite, couleur, metier, boisson, animal = 0, 1, 2, 3, 4

# et leurs valeurs
anglais, espagnol, islandais, norvegien, slovene = 0, 1, 2, 3, 4
rouge, vert, bleu, jaune, blanc = 0, 1, 2, 3, 4
diplomate, ingenieur, medecin, sculpteur, violoniste = 0, 1, 2, 3, 4
cafe, eau, lait, jusorange, the = 0, 1, 2, 3, 4
ane, cheval, chien, renard, zebre = 0, 1, 2, 3, 4

attributs = [
    ["anglais", "espagnol", "islandais", "norvégien", "slovène"],
    ["rouge", "vert", "bleu", "jaune", "blanc"],
    ["diplomate", "ingénieur", "médecin", "sculpteur", "violoniste"],
    ["café", "eau", "lait", "jus d'orange", "thé"],
    ["âne", "cheval", "chien", "renard", "zèbre"]
] 

Pour $(i,j,k)\in \{0,1,2,3,4\}^3$ notons $q_{i,j,k}$ la proposition " le $j^\text e$ attribut de la $i^\text e$ maison est $k$ " (les numérotations commencent à $0$).

Ainsi $q_{2,3,4}$ signifie " le propriétaire de la 2 $\!{}^\text{e}$ maison (en fait la 3 $\!{}^\text{e}$) boit du thé"

On cherche une affectation des variables propositionnelles $p_\ell=q_{i,j,k}$, où $\ell=1+i+5\times j+5^2\times k\in\{1,2,\ldots,125\}$, qui respecte les 14 conditions imposées. 

In [6]:
def q(maison,attribut,valeur):
    return 1 + maison + 5 * attribut + 25 * valeur

# formule à satisfaire :
f = []

# ajout à f des différentes clauses :

# chaque attribut de chaque maison a exactement une valeur :
for j in range(5):
    for i in range(5):
        exactementUn(f,{q(i,j,k) for k in range(5)})
       
# Chaque valeur possible de chaque attribut est attibuée à une maison
for k in range(5):
    for j in range(5):
        auMoinsUn(f,{q(i,j,k) for i in range(5)})

def sontVoisins(attribut1,valeur1,attribut2,valeur2):
    implique(f,q(0,attribut1,valeur1),q(1,attribut2,valeur2))
    for i in range(1,4):
        implique(f,q(i,attribut1,valeur1),{q(i+1,attribut2,valeur2),q(i-1,attribut2,valeur2)})    
    implique(f,q(4,attribut1,valeur1),q(3,attribut2,valeur2))

for i in range(5):
    
    # 1. l'anglais habite la maison rouge
    implique(f,q(i,nationalite,anglais),q(i,couleur,rouge))
    
    # 2. l'espagnol a un chien :
    implique(f,q(i,nationalite,espagnol),q(i,animal,chien))
    
    # 3. l'islandais est ingénieur :
    implique(f,q(i,nationalite,islandais),q(i,metier,ingenieur))
    
    # 4. l'occupant de la maison verte boit du café : 
    implique(f,q(i,couleur,vert),q(i,boisson,cafe))

    # 5. la maison verte est située immédiatement à gauche de la blanche,
    if i < 4: implique(f, q(i, couleur, vert), q(i + 1, couleur, blanc))
    else: faux(f,q(4,couleur,vert))
    
    # 6. le sculpteur possède un ane :
    implique(f,q(i,metier, sculpteur),q(i,animal,ane))

    # 7. le diplomate habite la maison jaune :
    implique(f,q(i,metier,diplomate),q(i,couleur,jaune))

# 8. le norvégien habite la première maison à gauche :
vrai(f,q(0,nationalite,norvegien))

# 9. le médecin habite la maison voisine de celle où demeure le propriétaire du renard,
sontVoisins(metier,medecin,animal,renard)

# 10. la maison du diplomate est voisine de celle où il y a un cheval :
sontVoisins(metier,diplomate,animal,cheval)

# 11. on boit du lait dans la maison du milieu
vrai(f,q(2,boisson,lait))

for i in range(5):
    
    # 12. le Slovène boit du thé :
    implique(f,q(i,nationalite,slovene),q(i,boisson,the))

    # 13. le violoniste boit du jus d'orange :
    implique(f,q(i,metier,violoniste),q(i,boisson,jusorange))
    
# 14. le norvégien demeure à côté de la maison bleue
sontVoisins(nationalite,norvegien,couleur,bleu)

In [7]:
d=dpll(deepcopy(f),125)

for i in range(5):
    for j in range(5):
        for k in range(5):
            if d[q(i,j,k)]:
                print('maison {} : {}'.format(i + 1,attributs[j][k]))    

maison 1 : norvégien
maison 1 : jaune
maison 1 : diplomate
maison 1 : eau
maison 1 : renard
maison 2 : slovène
maison 2 : bleu
maison 2 : médecin
maison 2 : thé
maison 2 : cheval
maison 3 : anglais
maison 3 : rouge
maison 3 : sculpteur
maison 3 : lait
maison 3 : âne
maison 4 : islandais
maison 4 : vert
maison 4 : ingénieur
maison 4 : café
maison 4 : zèbre
maison 5 : espagnol
maison 5 : blanc
maison 5 : violoniste
maison 5 : jus d'orange
maison 5 : chien


Pour la solution trouvée, l'islandais a le zèbre.

Vérifions qu'il n'y a pas d'autres solutions.

In [8]:
f_ = deepcopy(f)

f_.append({-q(i,j,k) 
               for i in range(5)
               for j in range(5)
               for k in range(5) if d[q(i,j,k)]})

print(dpll(f_,125))

None


## 2. [Sudoku](https://fr.wikipedia.org/wiki/Sudoku)

Une grille de Sudoku $t=(t_{i, j})_{i,j=0,\ldots,8}$ est représentée par une chaine de caractères 

In [9]:
exempleSudoku = """
    ..8.5....      
    .4....3..
    ......1..
    .7.3.....      
    ....2..8.
    1......5.
    ..57...4.
    ...1..6..
    2........
""" 

que l'on transformera en matrice d'entiers :

In [10]:
def matriceOfChaine(t):
    return [list(map(lambda x:0 if x == '.' else int(x),l)) for l in list(map(list,t.split()))]

In [11]:
matriceOfChaine(exempleSudoku)

[[0, 0, 8, 0, 5, 0, 0, 0, 0],
 [0, 4, 0, 0, 0, 0, 3, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 7, 0, 3, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 2, 0, 0, 8, 0],
 [1, 0, 0, 0, 0, 0, 0, 5, 0],
 [0, 0, 5, 7, 0, 0, 0, 4, 0],
 [0, 0, 0, 1, 0, 0, 6, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0]]

Pour $(i,j,k)\in \{0,\ldots,8\}^2\times \{1,\ldots,9\}$ notons $q_{i,j,k}$ la proposition $t_{i,j}=k$.

On cherche une affectation des variables propositionnelles $p_\ell=q_{i,j,k}$, où $\ell=1+i+9\times j+9^2\times(k-1)\in\{1,2,\ldots,729\}$, qui respecte les conditions imposées par les règles du Sudoku. 

In [12]:
def solveSudoku(t):

    def q(i,j,k):
        return 1 + i + 9 * j + 81 * (k - 1)
    
    m = matriceOfChaine(t)
    
    # formule à satisfaire :
    f = []

    # éléments déjà placés
    for i in range(9):
        for j in range(9):
            if m[i][j]:
                vrai(f,q(i,j,m[i][j]))
                  
    # contraintes de cases
    for i in range(9):
        for j in range(9):
            exactementUn(f,[q(i,j,k) for k in range(1,10)])
            
    # contraintes de lignes
    for i in range(9):
        for k in range(1,10):
            exactementUn(f,[q(i,j,k) for j in range(9)])
            
    # contraintes de colonnes
    for j in range(9):
        for k in range(1,10):
            exactementUn(f,[q(i,j,k) for i in range(9)])
     
    # contraintes de régions
    for u in range(3):
        for v in range(3):
            for k in range(1,10):
                exactementUn(f,[q(3 * u + i, 3 * v + j, k)  for i in range(3) for j in range(3)])
    
    d = dpll(f,9**3)
    
    for i in range(9):
        for j in range(9):
            for k in range(1,10):
                if d[q(i,j,k)]:
                    m[i][j] = k
                
    return '\n'.join([' '.join(list(map(str,m[i]))) for i in range(9)])
    

### Test

In [13]:
print(solveSudoku(exempleSudoku))

3 2 8 9 5 1 4 7 6
7 4 1 2 8 6 3 9 5
9 5 6 4 7 3 1 2 8
5 7 9 3 1 8 2 6 4
4 6 3 5 2 7 9 8 1
1 8 2 6 9 4 7 5 3
6 1 5 7 3 2 8 4 9
8 9 7 1 4 5 6 3 2
2 3 4 8 6 9 5 1 7


## 3. Le [problème des 8 dames](https://fr.wikipedia.org/wiki/Probl%C3%A8me_des_huit_dames)


Pour $(i,j)\in \{0,\ldots,7\}^2$ notons $q_{i,j}$ la proposition " la case $i,j$ contient une reine ".

On cherche une affectation des $p_\ell=q_{i,j}$, où $\ell=1+i+8\times j\in\{1,2,\ldots,64\}$.

In [14]:
def q(i,j):
    return 1 + i + 8 * j 

f= []

# lignes et colonnes   
for i in range(8):
    exactementUn(f,[q(i,j) for j in range(8)])
    exactementUn(f,[q(j,i) for j in range(8)])

# diagonales
for i in range(8):
    for j in range(8):
        for k in range(8):
            for l in range(8):
                if abs(l - j) == abs(k - i):
                    auPlusUn(f,[q(i,j),q(k,l)])  

d = dpll(f,64)

m = [0 for j in range(8)]

for i in range(8):
    for j in range(8):
        if d[q(i,j)]:
            m[i] = j
            
print('\n'.join('. ' * m[i] + 'X ' + '. ' * (7 - m[i]) for i in range(8)))

X . . . . . . . 
. . . . . . X . 
. . . . X . . . 
. . . . . . . X 
. X . . . . . . 
. . . X . . . . 
. . . . . X . . 
. . X . . . . . 
